# 北京空气质量分析与预测报告 (2010-2014)
# Air Quality Analysis & Prediction Report

**作者 / Author**: edg663  
**日期 / Date**: 2025  

## 1. 项目背景
本项目旨在分析北京 2010-2014 年的 PM2.5 数据，并尝试利用气象数据（温度、气压、风速）预测 PM2.5 浓度。
我们将从基础的线性回归升级为 **随机森林 (Random Forest)** 模型，并引入 **时间滞后特征 (Lag Features)** 以提升预测精度。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

import platform
system_name = platform.system()

if system_name == "Windows":
    plt.rcParams['font.sans-serif'] = ['SimHei']
elif system_name == "Darwin":
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
else:
    plt.rcParams['font.sans-serif'] = ['WenQuanYi Micro Hei']

plt.rcParams['axes.unicode_minus'] = False
sns.set(style="whitegrid", font=plt.rcParams['font.sans-serif'][0])

print(f"当前系统: {system_name}, 中文字体设置完成。")

In [ ]:
current_dir = Path.cwd()
project_root = current_dir.parent
data_path = project_root / "data" / "cleaned" / "air_quality_clean.csv"

try:
    df = pd.read_csv(data_path, index_col=0, parse_dates=True)
    print(f"成功读取数据，共 {len(df)} 条记录。")
except FileNotFoundError:
    print(f"错误：未找到文件 {data_path}。请先运行 src/test_data_cleaning.py")

df.head()

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(df.index, df['pm2.5'], color='steelblue', linewidth=0.5, alpha=0.7)
plt.title("北京 PM2.5 浓度随时间变化趋势 (2010-2014)", fontsize=14)
plt.xlabel("日期")
plt.ylabel("PM2.5 (ug/m3)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 6))
corr = df[['pm2.5', 'TEMP', 'PRES', 'Iws']].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("变量相关性热力图")
plt.show()

In [ ]:
print("正在构造特征...")
df['hour'] = df.index.hour
df['month'] = df.index.month
df['pm25_lag1'] = df['pm2.5'].shift(1)
df['pm25_lag24'] = df['pm2.5'].shift(24)
df['pm25_rolling_24'] = df['pm2.5'].rolling(window=24).mean()
df_model = df.dropna()
print(f"特征构造完成。最终用于建模的数据量: {len(df_model)}")
df_model.head(3)

In [ ]:
features = ['TEMP', 'PRES', 'Iws', 'hour', 'month', 'pm25_lag1', 'pm25_lag24', 'pm25_rolling_24']
X = df_model[features]
y = df_model['pm2.5']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

print("正在训练随机森林模型 (Random Forest)... 可能需要几秒钟")
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)
rf.fit(X_train, y_train)
print("模型训练完成！")

In [ ]:
y_pred = rf.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("-" * 30)
print(f"测试集 R² 分数: {r2:.4f}")
print(f"均方误差 (MSE): {mse:.2f}")
print("-" * 30)

plt.figure(figsize=(15, 6))
plt.plot(y_test.index[-200:], y_test.values[-200:], label='真实值 (Actual)', color='black', alpha=0.6)
plt.plot(y_test.index[-200:], y_pred[-200:], label='预测值 (Predicted)', color='red', linestyle='--', alpha=0.8)
plt.title(f"随机森林预测效果对比 (R²={r2:.3f})", fontsize=14)
plt.legend()
plt.show()

importances = pd.Series(rf.feature_importances_, index=features).sort_values()
plt.figure(figsize=(10, 6))
importances.plot(kind='barh', color='teal')
plt.title("随机森林模型 - 特征重要性排序", fontsize=14)
plt.xlabel("重要性系数")
plt.tight_layout()
plt.show()

## 结论 / Conclusion

通过引入时间滞后特征（Lag Features）并使用随机森林模型，我们取得了显著的成果：

1. **模型性能**：R² 达到了 **0.94** 左右，远超传统线性回归模型 (R² ~0.11)。
2. **关键因子**：特征重要性图表显示，**上一小时的 PM2.5 (pm25_lag1)** 是最重要的预测变量，这验证了空气质量具有极强的持续性和自相关性。
3. **应用价值**：该模型可以较好地预测未来 1 小时的空气质量趋势，具有实际预警价值。